In [ ]:
!pip install opencv-python --user

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from glob import glob
from PIL import Image
from numpy import asarray

In [ ]:
dirct ='../Brain-Tumor-Prediction'

In [ ]:
import os
path = os.listdir(dirct+'/Brain_Tumor_Data')
classes = {'no_tumor':0, 'pituitary_tumor':1}

In [ ]:
no_tumor_frompath= dirct+'/Brain_Tumor_Data/Training/no_tumor'
no_tumor_topath = dirct+'/Brain_Tumor_Data/Agumented_Data/Training/no_tumor/'
no_tumor_images=glob(no_tumor_frompath+'/*')
c=0
for path in no_tumor_images:
  img=Image.open(path)
  img=img.resize((224,224))
  img.save(no_tumor_topath+str(c)+'.jpg')
  c+=1
  mirimg=img.transpose(Image.FLIP_LEFT_RIGHT)
  mirimg.save(no_tumor_topath+str(c)+'.jpg')
  c+=1
  rotimg=img.rotate(90,Image.NEAREST, expand = 1)
  rotimg.save(no_tumor_topath+str(c)+'.jpg')
  c+=1
  mirrotimg=rotimg.transpose(Image.FLIP_LEFT_RIGHT)
  mirrotimg.save(no_tumor_topath+str(c)+'.jpg')
  c+=1

In [ ]:
frompath = dirct+'Brain_Tumor_Data/Training/pituitary_tumor'
topath = dirct+'Brain_Tumor_Data/Agumented_Data/Training/pituitary_tumor/'
images=glob(frompath+'/*')
c=0
for path in images:
  img=Image.open(path)
  img=img.resize((224,224))
  img.save(topath+str(c)+'.jpg')
  c+=1
  mirimg=img.transpose(Image.FLIP_LEFT_RIGHT)
  mirimg.save(topath+str(c)+'.jpg')
  c+=1
  rotimg=img.rotate(90,Image.NEAREST, expand = 1)
  rotimg.save(topath+str(c)+'.jpg')
  c+=1
  mirrotimg=rotimg.transpose(Image.FLIP_LEFT_RIGHT)
  mirrotimg.save(topath+str(c)+'.jpg')
  c+=1

In [ ]:
import cv2
X=[]
Y=[]
for cls in classes:
  pth = dirct+'/Brain_Tumor_Data/Agumented_Data/Training/'+cls
  for j in os.listdir(pth):
    img = cv2.imread(pth+'/'+j)   
    # img = cv2.resize(img,dsize=(224,224))
    X.append(img)
    Y.append(classes[cls])

In [ ]:
plt.imshow(X[0],cmap='gray')

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=40,test_size=0.20)
print ("Shape of an image in x_train: ", x_train[0].shape)
print ("Shape of an image in x_test: ", x_test[0].shape)

In [ ]:
from sklearn import preprocessing
import tensorflow as tf
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)
y_train = np.array(y_train)
x_train = np.array(x_train)
y_test = np.array(y_test)
x_test = np.array(x_test) 
print("x_train Shape: ", x_train.shape) 
print("x_test Shape: ", x_test.shape)
print("y_train Shape: ", y_train.shape) 
print("y_test Shape: ", y_test.shape)

In [ ]:
from keras.applications import vgg16

img_rows,img_cols=224,224

vgg=vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows,img_cols,3))

In [ ]:
for layer in vgg.layers:
  layer.trainable=False


In [ ]:
from keras.layers.pooling.global_average_pooling2d import GlobalAveragePooling2D
def lw(bottom_model,num_classes):
  top_model=bottom_model.output
  top_model=GlobalAveragePooling2D()(top_model)
  top_model=Dense(1024,activation='relu')(top_model)
  top_model=Dense(1024,activation='relu')(top_model)
  top_model=Dense(512,activation='relu')(top_model)
  top_model=Dense(num_classes,activation='softmax')(top_model)
  return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.models import Model

In [ ]:
import tempfile

def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = {'model_str': model_str}
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__


In [ ]:
make_keras_picklable()

In [ ]:
num_classes=2
FC_Head=lw(vgg,num_classes)
model=Model(inputs=vgg.input,outputs=FC_Head)
print(model.summary())

In [ ]:
from tensorflow.keras.models import Model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test),verbose=1,initial_epoch=0)


In [ ]:
# from sklearn.model_selection import KFold
# kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5 folds for validation
# histories = []  # List to store training history for each fold
# scores = []  # List to store accuracy scores for each fold

# for fold, (train_index, test_index) in enumerate(kf.split(X)):
#     print(f"Fold {fold + 1}")

#     x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
#     y_train, y_test = np.array(Y)[train_index], np.array(Y)[test_index]

#     # Convert to NumPy arrays and preprocess labels
#     from sklearn import preprocessing
#     import tensorflow as tf

#     le = preprocessing.LabelEncoder()
#     y_train = le.fit_transform(y_train)
#     y_test = le.fit_transform(y_test)
#     y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
#     y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

#     y_train = np.array(y_train)
#     x_train = np.array(x_train)
#     y_test = np.array(y_test)
#     x_test = np.array(x_test)

#     # Train the model for each fold
#     history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), verbose=1)
    
#     # Save training history and accuracy score
#     histories.append(history)
#     score = model.evaluate(x_test, y_test, verbose=0)
#     scores.append(score[1])  # Accuracy is at index 1 in `score`

# # Display K-Fold scores
# print(f"Fold scores: {scores}")
# print(f"Average accuracy: {np.mean(scores)}")

# # Visualize training curves for each fold
# plt.figure(figsize=(12, 6))

# for i, history in enumerate(histories):
#     plt.plot(history.history['val_accuracy'], label=f'Fold {i+1} Validation Accuracy')

# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Accuracy Evolution Across Folds')
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
plt.figure(1, figsize=(20, 8))

# Plot for Loss
plt.subplot(1, 2, 1)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.grid(True)
plt.legend()

# Plot for Accuracy
plt.subplot(1, 2, 2)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.grid(True)
plt.legend()

# Install seaborn if not already installed
!pip install seaborn

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

# Predict labels for the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions into a single class


# Display the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Tumor", "Pituitary Tumor"], yticklabels=["No Tumor", "Pituitary Tumor"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Display the classification report (precision, recall, F1-score)
print("Classification Report:\n", classification_report(np.argmax(y_test, axis=1), y_pred_classes))

# Display accuracy score
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_classes)
print("Accuracy Score: ", accuracy)

In [ ]:
import pickle
filename= 'brain_tumor_model.pkl'

In [ ]:

pickle.dump(model,open(filename,'wb'))

In [ ]:
load_model = pickle.load(open(filename,'rb'))
#load_model.predict(x_test)

In [ ]:
# MLP for Pima Indians Dataset Serialize to JSON and HDF5
from tensorflow.keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")

In [ ]:
loaded_model.predict(x_test)

In [ ]:
dec= {0: 'No Tumor', 1:'Positive Tumor'}

In [ ]:
plt.figure(figsize=(18,12))
p=os.listdir('Brain_Tumor_Data/Training/pituitary_tumor')
c=1
predict_img=[]
for i in os.listdir('Brain_Tumor_Data/Training/pituitary_tumor/')[:16]:
  plt.subplot(4,4,c)

  img=cv2.imread('Brain_Tumor_Data/Training/pituitary_tumor/'+i)
  img1 = cv2.resize(img,(224,224))
  predict_img.append(img1)
  p= load_model.predict(np.array(predict_img))
  result = np.argmax(p, axis =1)
  plt.title(dec[result[c-1]])
  plt.imshow(img,cmap='gray')
  plt.axis('off')
  c+=1

In [ ]:
plt.figure(figsize=(18,12))
p=os.listdir('Brain_Tumor_Data/Training/no_tumor')
c=1
predict_img=[]
for i in os.listdir('Brain_Tumor_Data/Training/no_tumor/')[:16]:
  plt.subplot(4,4,c)

  img=cv2.imread('Brain_Tumor_Data/Training/no_tumor/'+i)
  img1 = cv2.resize(img,(224,224))
  predict_img.append(img1)
  p= model.predict(np.array(predict_img))
  result = np.argmax(p, axis =1)
  plt.title(dec[result[c-1]])
  plt.imshow(img,cmap='gray')
  plt.axis('off')
  c+=1